# Detection of Gravitational waves using Echo state networks for space based detectors (LISA)

In this notebook we intend to build a machine learning based model (using ESN) to detect gravitational waves embedded in colored Gaussian noise generated using the power spectral density of LISA detectors.

@author: Ramit Dey, Turmoli Neogi

In [ ]:
#import packages

import sys, os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.python.ops import math_ops
#from ESN import EchoStateRNNCell
import matplotlib.pyplot as plt
from matplotlib import colors
from scipy.sparse import *
from scipy.sparse.linalg import *


from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import GlobalMaxPooling2D
#from tensorflow.keras.utils import np_utils
import np_utils
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

# random numbers
random_seed = np.frombuffer(os.urandom(4), dtype=np.uint32)[0]
print("seed: ", random_seed)

# Load Data

In [ ]:
#data generator for loading the data in batches

class DataGenerator(keras.utils.Sequence):
    'data generator function for loading the GW waveforms'
    
    def __init__(self, list_IDs, labels, batch_size=32, n_channels=1,t_len=160000, n_classes=2, shuffle=True):
        'Initialization'
        self.batch_size = batch_size
        self.labels = labels
        self.t_len=t_len
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size,self.t_len, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.load('scratch/LISA noise/ns_' + ID + '.npy')

            # Store class
            y[i] = self.labels[ID]

        return X, keras.utils.to_categorical(y, num_classes=self.n_classes)

In [ ]:
#data labels given as targets in the output for training the model

ID_list=[i for i in range(0,4000)]


#creating labels

a=[0 for i in range(0,2000)] #noise

b=[1 for i in range(0,2000)] #data

#label list

label_list=np.concatenate((np.array(a),np.array(b)))

# EMN classifier

In [ ]:
""" Echo State Network """
class reservoir_layer(object):
	def __init__(self, rng, n_in, n_res, IS, SR, sparsity, leakyrate, use_bias=False):
		self.n_in = n_in
		self.n_res = n_res #
		self.IS = IS #Input scaling
		self.SR = SR #spectral radius
		self.sparsity = sparsity
		self.leakyrate = leakyrate
		self.use_bias = use_bias
		self.W_in = 2*np.array(np.random.random(size=(n_res, n_in)))-1
		W_res_temp = sp.sparse.rand(self.n_res, self.n_res, self.sparsity)
		vals, vecs = sp.sparse.linalg.eigsh(W_res_temp, k=1)
		self.W_res = self.SR * W_res_temp / vals[0]
		b_bound = 0.1
		self.b = 2*b_bound*np.random.random(size=(self.n_res, 1))-b_bound


#function creating the echo states

	def update_states(self, data, dataset,string):
		n_forget_steps = 0
		nums_sample = np.shape(data)[0]
		nums_frame = np.shape(data)[1]
		echo_states = np.empty((nums_sample,(nums_frame-n_forget_steps), self.n_res), np.float32)
		for i_sample in range(nums_sample):
			series = data[i_sample]
			#print ('create echo-states of %4d-th %s sample in s'(i_sample, string, dataset))
			collect_states = np.zeros((nums_frame-n_forget_steps, self.n_res))
			x = np.zeros((self.n_res, 1))
			for t in range(nums_frame):
				#print ("sample %03d for %03d th time stamp processed ......" % (i_sample+1, t+1))
				u_t = np.asarray([series[t,:]]).T
				if self.use_bias:
					xUpd =  np.tanh(np.dot(self.W_in, self.IS*u_t) + np.dot(self.W_res.toarray(), x) + self.b)
				else:        
					xUpd =  np.tanh(np.dot(self.W_in, self.IS*u_t) + np.dot(self.W_res.toarray(), x))
				x = (1-self.leakyrate)*x + self.leakyrate*xUpd
				if t >= n_forget_steps:
					collect_states[t-n_forget_steps,:] = x.T
			collect_states = np.asarray(collect_states)
			echo_states[i_sample] = collect_states
		return echo_states


# create teh echo matrix

def run_loading(n_res, IS, SR, SP):
	train_x_lst,test_x_lst, train_y, test_y = split
	train_x=np.array(train_x_lst)
 	
	train_x=train_x.reshape(-1,16384,1)

 	
	test_x=np.array(test_x_lst)
	test_x=test_x.reshape(-1,16384,1)

 
 	#test_x=np.array(test_x)

	nums_train, len_series =  np.shape(train_x)[0], np.shape(train_x)[1]
	nums_test = np.shape(test_x)[0]
	rng = np.random.RandomState(1882517)
	n_res = n_res 
	IS = IS
	SR = SR
	SP = SP #sparsity
	LK = 1
	escnn = reservoir_layer(rng, n_in=1, n_res=n_res, IS=IS, SR=SR, sparsity=SP, leakyrate=LK, use_bias=False)
	train_echoes = escnn.update_states(train_x, dataset_name, 'train')
	test_echoes = escnn.update_states(test_x, dataset_name, 'test')
	return train_echoes, train_y, test_echoes, test_y, dataset_name, n_res, len_series, IS, SR, SP 


def transfer_labels(labels):
	indexes = np.unique(labels)
	num_classes = indexes.shape[0]
	num_samples = np.shape(labels)[0]
	for i in range(num_samples):
		new_label = np.argwhere(indexes == labels[i])[0][0]
		labels[i] = new_label
	return labels, num_classes

## Model parameters


In [ ]:
# these are the parameters defining the EMN model

number_res=8   #32 #reservoir size
IS=0.1
SR=0.9
SP=0.7
nb_epoch =50
batch_size=5
nb_filter=40  #originally 120
# ratio=[0.6,0.7]
ratio=[0.2,0.2]

nums_class=2 #noise + types of GW present on the data stream
dataset_name='GW'

In [ ]:
###

train_echoes, train_y, test_echoes, test_y, dataset_name, n_res, len_series, IS, SR, SP = run_loading(n_res =number_res,IS=IS,SR=SR,SP=SP)


In [ ]:
####

nb_class = nums_class
nb_sample_train = np.shape(train_echoes)[0]
nb_sample_test = np.shape(test_echoes)[0]
test_data = np.reshape(test_echoes,(nb_sample_test, 1, len_series, n_res))

L_train = [x_train for x_train in range(nb_sample_train)]
np.random.shuffle(L_train)
train_data = np.zeros((nb_sample_train, 1, len_series, n_res))
#train_label = np.zeros((nb_sample_train, nb_class))
for m in range(nb_sample_train):
 	train_data[m,0,:,:] = train_echoes[L_train[m],:,:]
# 	train_label[m,:] = train_y[L_train[m],:]

## NN parameters

In [ ]:
input_shape = (1, len_series, n_res)

nb_row=[np.int(ratio[0]*len_series),np.int(ratio[1]*len_series)]
nb_col = input_shape[2]

kernel_initializer = 'lecun_uniform'
activation = 'relu'
padding = 'valid'
strides = (1, 1)
data_format='channels_first'
optimizer = Adam(learning_rate=1e-4)
loss = ['binary_crossentropy', 'categorical_crossentropy']
verbose = 1
#model
input = Input(shape = input_shape)
convs = []

# Run model

In [ ]:
#CNN part of the EMN model

for j in range(len(nb_row)):
	conv = Conv2D(nb_filter, (nb_row[j], nb_col), kernel_initializer = kernel_initializer, activation = activation, 
	padding = padding, strides = strides, data_format = data_format)(input)
	conv = GlobalMaxPooling2D(data_format = data_format)(conv)
	convs.append(conv)
    
body_feature = concatenate(convs,name='concat_layer')
#body_feature = Dense(64, kernel_initializer = kernel_initializer, activation = activation)(body_feature)
#body_feature = Dense(128, kernel_initializer = kernel_initializer, activation = activation)(body_feature)
body_feature = Dropout(0.4)(body_feature)
output = Dense(nb_class, kernel_initializer = kernel_initializer, activation = 'softmax',name = 'dense_output')(body_feature)
model = Model(inputs=input, outputs = output)
model.summary()
model.compile(optimizer = optimizer, loss = loss[1], metrics = ['accuracy'])


In [ ]:
history = model.fit(train_data.tolist(), train_y, batch_size = batch_size, 
	epochs = nb_epoch,verbose = verbose)